In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

## First stage AGC

We'll use this first stage to make coarse adjustments to our analogue gain --- giving feedback to our [variable gain amplifier (VGA)](https://www.nooelec.com/store/downloads/dl/file/id/103/product/334/vega_datasheet_revision_1.pdf).

The RFSoC's Data Converters give us the option of monitoring two unique thresholds per channel. These threshold flags are all we need to drive our first AGC algorithm.

While there are a couple of programmable modes these thresholds can use, we'll use their "hysteresis" modes to minimise:

  1. The PL hardware utilisation, and...
  
  2. The reliance on round-trips to software via the Data Converter interrupts
  
This mode will track will raise the threshold flag as soon as a sample is above its given threshold, but fall only after N (programmable) consecutive samples have been below the threshold. Configuring two of these thresholds allows us to observe if our signal of interest is within a given range, while filtering our short dips within a single packet. If a sample is above our range, we will get immediate feedback. Then we know that the analog gain should be decreased, to protect the ADC and prevent saturation. Conversely, only after N (again, this is programmable) samples are below the lower threshold, we will get feedback that the analogue gain should be increased, increasing the SNR after the ADC.

As our threshold information is only binary, we cannot tell how far from the ideal gain we currently are. Therefore we have settle for a _linear_ response. We will linearly decrease the gain if the signal is too high, or linearly decrease the gain if the signal is too low.

A circuit for this is a simple up/down counter, controlled by interpreting the threshold flags.

The main complication here is that our control signal to the VGA, $V_c$, is not linear with respect to gain! We will need to compensate for this when generating the output of our AGC.

[Nooelec datasheet](https://www.nooelec.com/store/downloads/dl/file/id/103/product/334/vega_datasheet_revision_1.pdf) for the VeGA amplifier shows the following relationship between the control voltage and output gain (in decibels!).

![control voltage to gain plot for Nooelec VeGA](./assets/vega_response.svg)

Note that the middle section of the graph is approximately linear (again, this is in decibels!), which is an indication this compensation should be performed in the logarithmic domain. Having said that, it is cleary not a linear response ($y = m x + c$) over the full range. We will have to account for this as well.

## Hardware for Linear $\rightarrow$ Log conversion

First of all, we will need a circuit to convert our linear gain, $g$, (as predicted by our up/down counter) to a dB value, $G$. Mathematically this conversion is:

$G = 20 log_{10}(g)$

However, performing $log_{10}$ in digital arithmetic is very expensive. We should consider an approximation that is much cheaper. The most efficient (but numerically wreckless) approximation is:

$log_2(1+x) \approx x, \quad \forall\  0 \leq x \leq 1$

This approximation can be visualised as below:

In [ ]:
x = np.arange(0,1,0.001)
px.line(pd.DataFrame({'x':x, 'log':np.log2(1+x), 'approx': x}), 'x', ['log','approx'])

There is a noticable error in the middle of the range, but it is a good approximation for the required hardware of 1 adder.

Two things to note about this approximation are:

  1. The bound on the input range
  2. The logarithm base is 2 --- not 10, as needed for our dB calulation

We can address limitation 1 by arbitrary selection of our linear gain. Since $log_2(1+x) \approx x, \forall\  0 \leq x \leq 1$, let's define $x'$ such that $log_2(x')\approx x'-1, \forall\ 1 \leq x' \leq 2$. Since our circuit has full control over the range of $x$, this will not be a problem.

For limitation 2, let's consider the well-known rule for changing the base of logs:

$log_b(x) = \frac{log_a(x)}{log_a(b)}$

In our case, we want to transform $log_2(x')$ into $log_{10}(x')$. This results in the following:

$$
\begin{align}
log_{10}(x') = && \frac{log_2{x'}}{log_{2(10)}} \\
= && 0.301\ log_2(x') \\
\approx && 0.301\ (x'-1)
\end{align}
$$

Our gain ratio to decibel approximation then becomes:
$$
\begin{align}
20\times log_{10}(x') \approx && 20\times (0.301\ (x'-1)) \\
\approx && 6.02\ (x'-1) \\
\approx && 6.02\ x' - 6.02
\end{align}
$$

This approximation of some awkward digital arithmetic requires just one constant multiplication and one subtraction. Let's quickly visualise the error introduced by this approximation. 

In [ ]:
x = np.arange(1,2,0.001)
px.line(pd.DataFrame({'x':x, 'error':np.log2(x)-(x-1)}), 'x', 'error')

This approximation can incurr up to an $8\%$ error, but is well worth it in terms of FPGA utilisation.

## Compensating for non-linear gains

The next step is to compensate for the non-linearity of the the VGA's response to $V_c$.
We've recreated the response from the datasheet for a $1000\ MHz$ signal (roughly the middle of our expected use case) below.

In [ ]:
gain_db = [13,13,14,15,17,20,23,26,30,34,37,38,40,40,40.5,40.5,40.5]
vc      = [i*0.2 for i in range(len(gain_db))]

frame = pd.DataFrame({'Vc': vc, 'Gain (dB)':gain_db})

px.line(frame, 'Vc', 'Gain (dB)')

The "S" shape of this response is characteristic of a sigmoid function. We will need the inverse of this response in order to compensate for it, so let's try to model it by reading off the datasheet's graph and modelling it mathematically with the help of `scipy`. First, let's normalise the graph above --- we don't really care about the absolute values of decibel gain or $V_c$, just their relationship.

In [ ]:
normalise = lambda xs : [(x-min(xs))/(max(xs)-min(xs)) for x in xs]
gain_db = normalise(gain_db)
vc      = normalise(vc)
frame = pd.DataFrame({'Vc': vc, 'Gain (dB)':gain_db})

px.line(frame, 'Vc', 'Gain (dB)')

We know we're expecting some sort of sigmoid function, so let's define such a function  (and its inverse) with parameters for scaling and offsets in both axes. Since the y-axis is between 0 and 1, we know the y scaling factor and y offset ahead of time --- let's fix those values now too.

In [ ]:
np.seterr(divide='ignore')

def sigmoid(xs, x_scaling, y_scaling, x_offset, y_offset):
    return y_scaling * np.tanh(xs * x_scaling - x_offset) + y_offset

def inv_sigmoid(xs, x_scaling, y_scaling, x_offset, y_offset):
    y = (1/x_scaling) * (np.arctanh((xs - y_offset) / y_scaling) + x_offset)
    y[y == -np.inf] = 0
    y[y == np.inf] = 1
    return y

our_sigmoid     = lambda x, x_scaling, x_offset :     sigmoid(x, x_scaling, 0.5, x_offset, 0.5)
our_inv_sigmoid = lambda x, x_scaling, x_offset : inv_sigmoid(x, x_scaling, 0.5, x_offset, 0.5)

Given our known $V_c$ values and this sigmoid function, we can ask `scipy` to find the sigmoid function parameters that best represent our gain control. 

In [ ]:
popt, pcov = curve_fit(our_sigmoid, np.array(vc), gain_db, bounds=(0, [100, 100]))
frame['Gain Model (dB)'] = our_sigmoid(np.array(vc), popt[0], popt[1])
px.line(frame, 'Vc', ['Gain (dB)','Gain Model (dB)'])

As we can see from the visualisation, this model does seem to closely resemble our gain/$V_c$ response.

## Generating a calibration lookup table

Now that we have modelled this mathematically, we can implement this compensation with a simple lookup table of values using the inverse sigmoid function. The [output control signal is just 6 bits](https://www.nooelec.com/store/downloads/dl/file/id/103/product/334/vega_datasheet_revision_1.pdf), and since the response is very roughly linear, it is reasonable to truncate our gain (in dB) to 6 bits too. Therefore our lookup table will be just $2^6$ entries of 6 bit words. 

UPDATE: We're now using a 16 bit PMOD DAC, so to fit in one BRAM (26 kb), we'll have 2^11 entries of 16 bits each.

In fact, we can generate the contents of the lookup table ROM from this notebook:

In [ ]:
quantised_db = normalise(range(2**11))
quantised_vc = our_inv_sigmoid(np.array(quantised_db), popt[0], popt[1])
scaled_quantised_vc = [min(2**16-1, max(0,round(vc * (2**16-1)))) for vc in quantised_vc]

np.array(scaled_quantised_vc)

In [ ]:
for b in scaled_quantised_vc:
    print('0b{0:016b} :>'.format(int(b)))

That is all we need in order to design our first stage AGC. Let's take an extra moment to consider the error that our approximations have introducted. This assumes that our mathematical model is correct, of course!

## Estimating approximation errors

First of all, let's calculate the ideal $V_c$ values for the full input range of $1\rightarrow 2$

In [ ]:
test_gains_ratio = [1 + i/(2**10-1) for i in range(2**10)]
test_gains = np.array([20*np.log10(i) for i in test_gains_ratio])
test_gains_norm = test_gains / max (test_gains)
actual_vc = our_inv_sigmoid(test_gains_norm, popt[0], popt[1])

And now, calculate the $V_c$ values for our implementation

In [ ]:
est_const = 20/np.log2(10)
est_gains = np.array([est_const*i - est_const for i in test_gains_ratio])
est_gains = est_gains / max (test_gains)
est_vc = our_inv_sigmoid(est_gains, popt[0], popt[1])

We want to compare our system against a version that uses our $log_{10}$ approximation, but assumes that the amplifier's response is linear (in dB).

In [ ]:
est_lin_vc = est_gains

err_frame = pd.DataFrame({'Gain (ratio)': test_gains_ratio, 'Error log estimate':est_lin_vc - actual_vc, 'Error our estimate': est_vc - actual_vc})
px.line(err_frame, 'Gain (ratio)', ['Error log estimate', 'Error our estimate'])

From this graph, we can see that by including our compensation for the amplifier's control response we've got a better output by up to $16\%$ --- that's likely worth the extra 384 bits of memory required!

Note that the overall error of our system is less than that of our original $log_{10}$ estimation. This is a nice coincidence since that error was at a maximum in the mid-range --- where the amplifier's gain is the most linear.

1: https://www.nooelec.com/store/downloads/dl/file/id/103/product/334/vega_datasheet_revision_1.pdf "VeGA Datasheet"

2: https://www.xilinx.com/support/documentation/user_guides/ug579-ultrascale-dsp.pdf "UltraScale+ DSP block Datasheet"

3: https://www.wiley.com/en-nl/Arithmetic+Circuits+for+DSP+Applications-p-9781119206774 "Great book for DSP approximations"

4: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html "Docs for scipy curve fitting"

5: https://www.vlebooks.com/vleweb/Product/Index/40057?page=0 "SDR book including AGC algorithms"

6: https://www.eetimes.com/wireless-101-automatic-gain-control-agc/# "EETimes blog based on the book in [5]"